In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from PIL import Image

In [19]:
L = np.pi*2 * 2
T = 20

a  = 1e1
dx = 1e-2
dt = 1e-2

x = np.arange(0, L+dx, dx)
t = np.arange(0, T+dt, dt)

Nt, Nx = len(t), len(x)

def u_0(x):
    "Initial Condition"
    return np.cos(x)

def D2u(n):
    D2 = np.zeros((n, n))
    for i in range(1, n-1):
        D2[i, [i-1, i, i+1]] = [1, -2, 1]
    D2[0, [0, 1]]    = [-2, 2]
    D2[-1, [-2, -1]] = [2, -2]
    return 1/dx * D2

D2 = D2u(Nx)
I  = np.diag(np.ones(Nx))
u = np.zeros((Nt, Nx))

def F(ti):
    return a*D2

for i, ti in enumerate(t):
    if i == 0:
        u[0, :] = u_0(x)
    else:
        j  = i-1
        uj = u[j, :]
        tj = t[j]
        u[i, :] = np.linalg.solve((I - dt/2 * F(tj)), (I + dt/2 * F(ti))@uj)
    
    if np.max(np.abs(u[i, :])) > 1e2:
        print(f"Blow up at time {ti:.3f}s")
        break
    
    if i % 100 == 0:
        print(f"Time {ti:.3f}s, max(u) = {np.max(np.abs(u[i, :])):.3f}")
        



Time 0.000s, max(u) = 1.000
Time 1.000s, max(u) = 0.906
Time 2.000s, max(u) = 0.820
Time 3.000s, max(u) = 0.743
Time 4.000s, max(u) = 0.672
Time 5.000s, max(u) = 0.609
Time 6.000s, max(u) = 0.551
Time 7.000s, max(u) = 0.499
Time 8.000s, max(u) = 0.452
Time 9.000s, max(u) = 0.409
Time 10.000s, max(u) = 0.370
Time 11.000s, max(u) = 0.335
Time 12.000s, max(u) = 0.303
Time 13.000s, max(u) = 0.275
Time 14.000s, max(u) = 0.249
Time 15.000s, max(u) = 0.225
Time 16.000s, max(u) = 0.204
Time 17.000s, max(u) = 0.185
Time 18.000s, max(u) = 0.167
Time 19.000s, max(u) = 0.151
Time 20.000s, max(u) = 0.137


In [20]:
def save_gif_PIL(outfile, files, fps=5, loop=0):
    "Helper function for saving GIFs"
    imgs = [Image.open(file) for file in files]
    imgs[0].save(fp=outfile, format='GIF', append_images=imgs[1:], save_all=True, duration=round(1000/fps), loop=loop)

def make_plot(save=False, file_name=f"PDE_solution.png", ti=0):
    "Helper function for plotting"
    fig, ax = plt.subplots()
    ax.plot(x, u[i, :])
    ax.set_xlim(0, L)
    ax.set_ylim(-1.05, 1.05)
    ax.grid()
    ax.set_title(f"Time = {ti:.3f}s")
    if save:
        fig.savefig(file_name, bbox_inches="tight", pad_inches=0.1, dpi=100, facecolor="white")

In [21]:
import os
import shutil

folder_path="plots"
for item in os.listdir(folder_path):
    item_path = os.path.join(folder_path, item)
    try:
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)  # Remove file or symbolic link
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)  # Remove subdirectory and its contents
    except Exception as e:
        print(f"Failed to delete {item_path}. Reason: {e}")

In [22]:
files = []
fps   = 20
frame_indicies = np.linspace(0, Nt-1, int(fps*T), dtype=int)
for i in frame_indicies:
    ti = t[i]
    file = f"plots/PDE_sol_{ti:.5f}.png"
    make_plot(save=True, file_name=file, ti=ti)
    files.append(file)
    plt.close("all")
    
save_gif_PIL("PDE_solution.gif", files, fps=fps, loop=0)